## Installation und Einrichtung
Bevor du startest, stelle sicher, dass du die folgenden Pakete installiert hast. Führe in einer Jupyter-Notebook-Zelle Folgendes aus:

``pip install stable-baselines3 pandas matplotlib``

stable-baselines3: Für Reinforcement Learning.
pandas: Für die Datenverarbeitung.
matplotlib: Für die grafische Darstellung.
ccxt: Für die API-Anbindung an Kryptobörsen wie Binance.

##  Datenimport und Vorbereitung
In dieser Zelle holen wir die historischen Bitcoin-Daten:

In [13]:
import pandas as pd
import yfinance as yf

def fetch_bitcoin_data(start_date, end_date):
    """
    Ruft Bitcoin-Daten von Yahoo Finance für den angegebenen Zeitraum ab.

    Args:
        start_date (str): Startdatum im Format 'YYYY-MM-DD'.
        end_date (str): Enddatum im Format 'YYYY-MM-DD'.

    Returns:
        pd.DataFrame: DataFrame mit den historischen Kursdaten.
    """
    try:
        # Überprüfung der Datumsformate
        pd.to_datetime(start_date)
        pd.to_datetime(end_date)

        # Sicherstellen, dass start_date vor end_date liegt
        if start_date >= end_date:
            raise ValueError("Das Startdatum muss vor dem Enddatum liegen.")

        # Lade Daten von Yahoo Finance
        df = yf.download("BTC-USD", start=start_date, end=end_date, interval="1d")

        # Überprüfen, ob Daten zurückgegeben wurden
        if df.empty:
            raise ValueError("Keine Daten für den angegebenen Zeitraum verfügbar.")

        # Umbenennen der Spalten für Konsistenz
        df.rename(columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Volume": "volume"
        }, inplace=True)

        # Filtere die relevanten Spalten und setze den Zeitstempel als Index
        df = df[["open", "high", "low", "close", "volume"]]
        df.index.name = "timestamp"

        return df

    except Exception as e:
        print(f"Fehler beim Abrufen der Daten: {e}")
        return pd.DataFrame()

In [14]:
# Abrufen der Daten
data = fetch_bitcoin_data('2018-01-01', '2023-12-01')
print(data.head())  # Zeige die ersten Zeilen der Daten

[*********************100%***********************]  1 of 1 completed

Price               open          high           low         close  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
timestamp                                                            
2018-01-01  14112.200195  14112.200195  13154.700195  13657.200195   
2018-01-02  13625.000000  15444.599609  13163.599609  14982.099609   
2018-01-03  14978.200195  15572.799805  14844.500000  15201.000000   
2018-01-04  15270.700195  15739.700195  14522.200195  15599.200195   
2018-01-05  15477.200195  17705.199219  15202.799805  17429.500000   

Price            volume  
Ticker          BTC-USD  
timestamp                
2018-01-01  10291200000  
2018-01-02  16846600192  
2018-01-03  16871900160  
2018-01-04  21783199744  
2018-01-05  23840899072  


- **fetch_ohlcv**: Ruft Open, High, Low, Close, Volume (OHLCV) Daten ab.
- **timestamp**: Zeitstempel für jeden Tag.
- **close**: Schlusskurs, der für das Training verwendet wird

## Umgebung und Agent erstellen
Hier definieren wir die RL-Umgebung und den Agenten:

In [23]:
import gym
import numpy as np

class BitcoinTradingEnv(gym.Env):
    def __init__(self, data, window_size=30):
        super(BitcoinTradingEnv, self).__init__()
        self.data = data[['open', 'high', 'low', 'close', 'volume', 'sma_10', 'sma_30', 'ema_10', 'ema_30']].values
        self.window_size = window_size
        self.current_step = window_size

        # Action Space: Kaufen, Verkaufen, Halten
        self.action_space = gym.spaces.Discrete(3)

        # Observation Space: Fenstergröße x Feature-Anzahl
        n_features = self.data.shape[1]
        self.observation_space = gym.spaces.Box(
            low=0, high=1, shape=(window_size, n_features), dtype=np.float32
        )

    def reset(self):
        self.current_step = self.window_size
        return self._get_observation()

    def _get_observation(self):
        """
        Gibt ein Fenster der letzten `window_size` Tage für alle Features zurück.
        """
        window = self.data[self.current_step - self.window_size:self.current_step]
        normalized_window = (window - np.min(self.data, axis=0)) / (
            np.max(self.data, axis=0) - np.min(self.data, axis=0)
        )
        return normalized_window

    def step(self, action):
        """
        Führt eine Aktion aus und berechnet die Belohnung.
        """
        reward = 0  # Default-Belohnung

        # Abbruchbedingung bei Überschreiten der Datenlänge
        if self.current_step >= len(self.data) - 2:
            done = True
            return self._get_observation(), reward, done, {}

        price_now = self.data[self.current_step, 3]  # Schließen-Preis heute
        price_next = self.data[self.current_step + 1, 3]  # Schließen-Preis morgen

        # Aktion auswerten
        if action == 1:  # Kaufen
            reward = (price_next - price_now) - 0.001 * price_now  # Handelskosten
        elif action == 2:  # Verkaufen
            reward = (price_now - price_next) - 0.001 * price_now  # Handelskosten
        elif action == 0:  # Halten
            reward = -0.01  # Minimaler Verlust für Inaktivität

        # Nächster Schritt
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1

        return self._get_observation(), reward, done, {}


In [ ]:
# Teste die Beobachtung
env = DummyVecEnv([lambda: BitcoinTradingEnv(data, window_size=60)])  # 60-Tage-Fenster
obs = env.reset()
print("Beobachtung nach Reset:", obs.shape)  # Sollte (1, 60) sein, da `DummyVecEnv` die Dimension erweitert

NameError: name 'env' is not defined

In [19]:
def add_technical_indicators(data):
    """
    Fügt technische Indikatoren wie gleitende Durchschnitte dem DataFrame hinzu.

    Args:
        data (pd.DataFrame): Historische Kursdaten.

    Returns:
        pd.DataFrame: DataFrame mit zusätzlichen Indikatoren.
    """
    data['sma_10'] = data['close'].rolling(window=10).mean()
    data['sma_30'] = data['close'].rolling(window=30).mean()
    data['ema_10'] = data['close'].ewm(span=10, adjust=False).mean()
    data['ema_30'] = data['close'].ewm(span=30, adjust=False).mean()
    
    # Entferne NaN-Werte, die durch gleitende Fenster entstehen
    data = data.dropna()
    return data


## Training und Modell speichern
Führe diesen Code aus, um das Modell zu trainieren und zu speichern:

In [20]:
def evaluate_trading_model(model, env, n_eval_episodes=10):
    """
    Evaluiert die Leistung des Modells über eine bestimmte Anzahl von Episoden.

    # Args:
        model: Das trainierte RL-Modell.
        env: Die Handelsumgebung.
        n_eval_episodes (int): Anzahl der Episoden zur Evaluierung.

    Returns:
        tuple: Durchschnittliche Belohnung und Standardabweichung.
    """
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes, deterministic=True)
    print(f"Durchschnittliche Belohnung: {mean_reward} ± {std_reward}")
    return mean_reward, std_reward

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Schritt 1: Daten laden und vorbereiten
# Hier werden die historischen Bitcoin-Daten geladen und technische Indikatoren hinzugefügt.
data = fetch_bitcoin_data('2018-01-01', '2023-12-31')  # Daten von 2018 bis Ende 2023
data = add_technical_indicators(data)  # Füge technische Indikatoren wie gleitende Durchschnitte hinzu
if data.empty:
    raise ValueError("Die geladenen Daten sind leer. Überprüfen Sie den Zeitraum oder die API.")

# Schritt 2: Umgebung initialisieren
# Die Umgebung simuliert den Bitcoin-Handel, basierend auf den vorbereiteten Daten.
env = DummyVecEnv([lambda: BitcoinTradingEnv(data, window_size=60)])  # 60-Tage-Fenster

# Schritt 3: Modell laden oder neu erstellen
# Hier wird versucht, ein bereits gespeichertes Modell zu laden.
# Falls keins existiert, wird ein neues Modell erstellt.
try:
    model = PPO.load("bitcoin_trading_model.zip", env=env)  # Versuche, ein bestehendes Modell zu laden
    print("Modell erfolgreich geladen. Training wird fortgesetzt.")
except (FileNotFoundError, ValueError) as e:
    print(f"Kein bestehendes Modell gefunden oder Fehler beim Laden: {e}")
    model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./bitcoin_trading_logs/")  # Neues Modell erstellen

# Schritt 4: Training des Modells
# Das Modell wird für eine bestimmte Anzahl von Schritten trainiert. Es beginnt dort, wo es zuletzt aufgehört hat.
total_timesteps = 50000  # Anzahl der Trainingsschritte
model.learn(total_timesteps=total_timesteps, reset_num_timesteps=False)  # Fortlaufendes Training
print(f"Modell wurde für {total_timesteps} Schritte trainiert.")

# Schritt 5: Evaluierung des Modells
mean_reward, std_reward = evaluate_trading_model(model, env)

# Schritt 6: Modell speichern
# Das Modell wird immer gespeichert, um sicherzustellen, dass Fortschritte nicht verloren gehen.
model.save("bitcoin_trading_model.zip")
print("Modell wurde gespeichert.")

[*********************100%***********************]  1 of 1 completed
c:\Users\tlfin\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Modell erfolgreich geladen. Training wird fortgesetzt.
Logging to ./bitcoin_trading_logs/PPO_0
-------------------------------
| time/              |        |
|    fps             | 1102   |
|    iterations      | 1      |
|    time_elapsed    | 1      |
|    total_timesteps | 206848 |
-------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 719           |
|    iterations           | 2             |
|    time_elapsed         | 5             |
|    total_timesteps      | 208896        |
| train/                  |               |
|    approx_kl            | 1.4673278e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.569        |
|    explained_variance   | 0.065001905   |
|    learning_rate        | 0.0003        |
|    loss                 | 2.55e+05      |
|    n_updates            | 1010          |
|    policy_gradient_